In [2]:
import time
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour, PeriodicBehaviour, OneShotBehaviour, FSMBehaviour, State
from spade.message import Message
from spade.template import Template
import json
import asyncio

In [3]:
from datetime import datetime

today = datetime.today()
print("Today's date:", today)

Today's date: 2022-05-25 21:54:25.397893


In [4]:
today

datetime.datetime(2022, 5, 25, 21, 54, 25, 397893)

In [5]:
password="Secret777"

In [6]:
jid ="SystemAgent@jabbim.pl"

In [50]:
class MachineAgent(Agent):

    def __init__(self, jid: str, password: str, verify_security: bool, inputs, outputs, providers, magazine, production_time,item_prices,euro, stock):
        self.inputs=inputs
        self.outputs=outputs
        self.Providers=providers
        self.magazine=magazine
        self.production_time=production_time
        self.item_prices=item_prices
        self.euro=euro
        self.needed={item:self.inputs[item]-self.magazine[item] for item in self.inputs if self.magazine[item]<self.inputs[item]}
        self.start_production=True
        self.Stock=stock
        super().__init__(jid, password, verify_security=verify_security)

    class ProduceBehav(CyclicBehaviour):
        async def run(self):
            await asyncio.sleep(1)
            self.agent.needed={item:self.agent.inputs[item]-self.agent.magazine[item] for item in self.agent.inputs if self.agent.magazine[item]<self.agent.inputs[item]}
            if self.agent.start_production and len(self.agent.needed)==0:
                print("start_production: "+str(self.agent.jid))
                await asyncio.sleep(self.agent.production_time)
                for item in self.agent.inputs:
                    self.agent.magazine[item]=self.agent.magazine[item]-self.agent.inputs[item]
                for item in self.agent.outputs:
                    self.agent.magazine[item]=self.agent.magazine[item]+self.agent.outputs[item]

                self.agent.Stock.notify(str(self.agent.jid),datetime.today(),self.agent.magazine)
            #self.agent.start_production=False

    class BuyBehav(CyclicBehaviour):
        async def on_start(self):
            await asyncio.sleep(4)

        async def run(self):
            propositions_list=[]
            if len(self.agent.needed)>0:
                for item in self.agent.needed:
                    for Provider in self.agent.Providers[item]:
                        msg = Message(to=Provider)
                        msg.set_metadata("buy", "order")
                        msg.body = json.dumps({"item": item,"quantity":str(self.agent.needed[item])})
                        await self.send(msg)
                        print(Provider+" please "+item)
                        
                    
                    while len(propositions_list)<len(self.agent.Providers[item]):
                        msg = await self.receive(timeout=10)
                        propositions_list.append(msg)
                    
                    best_price=100000
                    best_proposition=None
                    for msg in propositions_list:
                        if msg.get_metadata("sell")=="proposition":
                            print("Message received with content: {}".format(msg.body))
                            body = json.loads(msg.body)
                            get_item = body["item"]
                            get_payment = int(body["payment"])
                            if get_payment<best_price:
                                best_proposition=msg
                                best_price=get_payment
                                
                    if best_proposition:
                        msg = Message(to=str(best_proposition.sender))
                        msg.set_metadata("buy", "payment")
                        if item in self.agent.needed.keys():
                            msg.body = json.dumps({"item": item, "quantity":str(self.agent.needed[item]),"payment":str(best_price)})
                            self.agent.euro-=best_price
                            await self.send(msg)
                            print("sending cash")

                            msg = await self.receive(timeout=10)

                            body = json.loads(msg.body)
                            get_item = body["item"]
                            get_quantity = int(body["quantity"])

                            self.agent.magazine[item]=self.agent.magazine[item]+get_quantity
                            self.agent.Stock.notify(str(self.agent.jid),datetime.today(),self.agent.magazine)
                            print("thank you " + str(best_proposition.sender))


    class SellBehav(CyclicBehaviour):
        async def run(self):
            await asyncio.sleep(1)
            msg = await self.receive(timeout=10)
            if msg:
                get_type=msg.get_metadata("buy")
                body = json.loads(msg.body)
                get_item = body["item"]
                get_quantity = int(body["quantity"])
                if get_type=="order":
                    if self.agent.magazine[get_item]>=get_quantity:
                        msg = Message(to=str(msg.sender))
                        msg.set_metadata("sell", "proposition")
                        msg.body = json.dumps({"item": get_item,"payment":str(self.agent.item_prices[get_item]*get_quantity)})
                        await self.send(msg)
                        print("proposing")
                    else:
                        msg = Message(to=str(msg.sender))
                        msg.set_metadata("sell", "decline")
                        msg.body = json.dumps({"item": get_item, "quantity":0})
                        await self.send(msg)
                        print("no item now")
                        self.agent.start_production=True
                        
                if get_type=="payment":
                    get_payment = int(body["payment"])
                    self.agent.euro+=get_payment
                    self.agent.magazine[get_item]=self.agent.magazine[get_item]-get_quantity
                    self.agent.Stock.notify(str(self.agent.jid),datetime.today(),self.agent.magazine)
                    msg = Message(to=str(msg.sender))
                    msg.set_metadata("sell", "product")
                    msg.body = json.dumps({"item": get_item,"quantity":str(get_quantity)})
                    await self.send(msg)
                    print("sending product")
                    
            else:
                print("waiting for sell")

    async def setup(self):
        print(f"MachineAgent started at {datetime.now().time()}")
        t_product = Template()
        t_product.set_metadata("sell", "product")
        t_decline = Template()
        t_decline.set_metadata("sell", "decline")
        t_proposition = Template()
        t_proposition.set_metadata("sell", "proposition")
        t_order = Template()
        t_order.set_metadata("buy", "order")
        t_payment = Template()
        t_payment.set_metadata("buy", "payment")
        self.p = self.ProduceBehav()
        self.add_behaviour(self.p)
        self.b = self.BuyBehav()
        self.add_behaviour(self.b,t_product | t_decline | t_proposition)
        self.s = self.SellBehav()
        self.add_behaviour(self.s,t_payment | t_order)

In [51]:
class Stock():

    def __init__(self,agent_magazines):
        self.agent_magazines=agent_magazines

    def notify(self,agent,timestamp,magazine):
        self.agent_magazines[agent].append((timestamp,magazine.copy()))

In [52]:
if __name__ == "__main__":
    stock=Stock({str.lower(jid+"/1"):[],str.lower(jid+"/2"):[],str.lower(jid+"/3"):[]})

    agent1 = MachineAgent(jid+"/1", password, True,{"coffe":2,"milk":1},{"cappucino":1},{"coffe":[],"milk":[jid+"/2",jid+"/3"]},{"coffe":40,"milk":0,"cappucino":0},2,{},200,stock)
    
    agent2 = MachineAgent(jid+"/2", password, True,{"magic":0},{"milk":1},{"milk":[jid+"/2"]},{"magic":0,"milk":0},3,{"milk":30},0,stock)
    
    agent3 = MachineAgent(jid+"/3", password, True,{"magic":0},{"milk":1},{"milk":[jid+"/3"]},{"magic":0,"milk":0},1,{"milk":50},0,stock)
  
    
    try:
        future = agent1.start()
        future.result()
        agent2.start()
        agent3.start()
        time.sleep(20)
    except Exception as e:
        print(e)
    finally:
        agent1.stop()
        agent2.stop()
        agent3.stop()

MachineAgent started at 23:08:29.444535
MachineAgent started at 23:08:30.121053
MachineAgent started at 23:08:30.236161
start_production: systemagent@jabbim.pl/2
start_production: systemagent@jabbim.pl/3
start_production: systemagent@jabbim.pl/3
SystemAgent@jabbim.pl/2 please milk
SystemAgent@jabbim.pl/3 please milk
no item now
proposing
Message received with content: {"item": "milk", "payment": "50"}
sending cash
sending product
thank you systemagent@jabbim.pl/3
SystemAgent@jabbim.pl/2 please milk
SystemAgent@jabbim.pl/3 please milk
proposing
start_production: systemagent@jabbim.pl/2
start_production: systemagent@jabbim.pl/3
start_production: systemagent@jabbim.pl/1
proposing
Message received with content: {"item": "milk", "payment": "30"}
Message received with content: {"item": "milk", "payment": "50"}
start_production: systemagent@jabbim.pl/3
SystemAgent@jabbim.pl/2 please milk
SystemAgent@jabbim.pl/3 please milk
proposing
proposing
Message received with content: {"item": "milk", "p

In [49]:
"milk" in agent1.needed.keys()

True

In [54]:
agent1.euro

60

In [53]:
stock.agent_magazines

{'systemagent@jabbim.pl/1': [(datetime.datetime(2022, 5, 25, 23, 8, 34, 457455),
   {'coffe': 40, 'milk': 1, 'cappucino': 0}),
  (datetime.datetime(2022, 5, 25, 23, 8, 37, 453677),
   {'coffe': 38, 'milk': 0, 'cappucino': 1}),
  (datetime.datetime(2022, 5, 25, 23, 8, 39, 458870),
   {'coffe': 38, 'milk': 1, 'cappucino': 1}),
  (datetime.datetime(2022, 5, 25, 23, 8, 42, 455953),
   {'coffe': 36, 'milk': 0, 'cappucino': 2}),
  (datetime.datetime(2022, 5, 25, 23, 8, 44, 460470),
   {'coffe': 36, 'milk': 1, 'cappucino': 2}),
  (datetime.datetime(2022, 5, 25, 23, 8, 47, 458105),
   {'coffe': 34, 'milk': 0, 'cappucino': 3}),
  (datetime.datetime(2022, 5, 25, 23, 8, 49, 463056),
   {'coffe': 34, 'milk': 1, 'cappucino': 3}),
  (datetime.datetime(2022, 5, 25, 23, 8, 52, 461755),
   {'coffe': 32, 'milk': 0, 'cappucino': 4})],
 'systemagent@jabbim.pl/2': [(datetime.datetime(2022, 5, 25, 23, 8, 34, 123822),
   {'magic': 0, 'milk': 1}),
  (datetime.datetime(2022, 5, 25, 23, 8, 38, 124091), {'magic'

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go

<ipython-input-1-b336346859d7>:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-b336346859d7>:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
es = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=es)

xs = list(range(30))
ys = [10000 * 1.07**i for i in xs]

fig = go.Figure(data=go.Scatter(x=xs, y=ys))
fig.update_layout(xaxis_title='Years', yaxis_title='$')

app.layout = html.Div(children=[
    html.H1(children='Factory Project implemented using SPADE'),
    dcc.Graph(figure=fig)])

if __name__ == '__main__':
  app.run(use_reloader=False)
  #  app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/May/2022 19:18:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 19:18:18] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_4_1m1653499004.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 19:18:18] "GET /_dash-component-suites/dash/deps/react@16.v2_4_1m1653499004.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 19:18:18] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_4_1m1653499004.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 19:18:18] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_4_1m1653499004.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 19:18:18] "GET /_dash-component-suites/dash/html/dash_html_components.v2_0_3m1653499004.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 19:18:18] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_4_0m1653499004.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 19:18:18] "GET /_dash-component-suites/dash/dash